### find a city

In [1]:
import requests
import sqlite3
import pandas as pd

# NOAA API token
api_token = 'rDZYpxCxcNrIUzeserZMcYDaaHCNPcFZ'

# connect to the DB
conn = sqlite3.connect('C:/Users/Mark Rozenberg/Foobal-Climate/Data/Main_DB.db')

In [5]:
# Query to list all tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

             name
0     goalscorers
1       shootouts
2       locations
3       countries
4        stations
5  noaa_countries
6         results
7         climate


### extract data for desired cities

In [4]:
pd.read_sql_query('''
SELECT count(distinct location_date_id) as count
FROM climate
''', conn)

,count
0,12539


In [55]:
pd.read_sql_query('''
SELECT location_date_id, count(*) as count
FROM climate
GROUP BY location_date_id
HAVING count > 1
order by count desc
limit 10
''', conn)

,location_date_id,count
0,CITY:JA000016_1958-05-28,1300
1,CITY:US360019_1949-06-19,1166
2,CITY:MX000020_1977-10-12,994
3,CITY:US080004_2019-06-19,896
4,CITY:AS000010_1973-03-16,756
5,CITY:AS000010_1973-03-13,756
6,CITY:AS000010_1973-03-11,752
7,CITY:AS000010_1973-03-24,748
8,CITY:MX000020_1968-03-10,705
9,CITY:MX000020_1977-10-23,656


In [5]:
pd.read_sql_query('''
SELECT count(distinct location_date_id) as count
FROM results
''', conn)

,count
0,28553


In [56]:
pd.read_sql_query('''
SELECT location_date_id, count(*) as count
FROM results
GROUP BY location_date_id
HAVING count > 1
order by count desc
limit 10
''', conn)

,location_date_id,count
0,None,16221
1,CITY:SG000001_1963-04-14,8
2,CITY:SG000001_1963-04-12,8
3,CITY:SG000001_1963-04-16,7
4,CITY:IV000002_1961-12-25,5
5,CITY:CF000001_1965-07-20,5
6,CITY:TS000012_1975-12-21,4
7,CITY:TH000005_1995-12-12,4
8,CITY:TH000005_1995-12-10,4
9,CITY:TH000005_1995-12-08,4


In [128]:
pd.read_sql_query('''
with part1 as(
select location_date_id,
max(case when datatype not in('err','err2') then 1 else 0 end) as valid,
max(case when datatype = 'err' then 1 else 0 end) as err_type1,
max(case when datatype = 'err2' then 1 else 0 end) as err_type2
from climate
group by 1
)
select sum(valid) as valid, sum(err_type1) as err_type1, sum(err_type2) as err_type2
from part1
''', conn)

,valid,err_type1,err_type2
0,20173,0,888


In [129]:
pd.read_sql_query('''
select count(distinct location_date_id)
FROM results
WHERE
location_id IS NOT NULL
and valid = 1
and location_date_id not in(select location_date_id from climate)
''', conn)

,count(distinct location_date_id)
0,5204


In [130]:
# Create a DataFrame from the results table with locationid
test = pd.read_sql_query('''
SELECT distinct date_ts, location_id
FROM results
WHERE
location_id IS NOT NULL
and valid = 1
and location_date_id not in(select location_date_id from climate)
order by date_ts
''', conn)
print(f"Number of rows in test: {len(test)}")

Number of rows in test: 5204


In [131]:
import requests
import sqlite3
import json

# Define the endpoint
endpoint = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'

# Set the headers with the API token
headers = {
    'token': api_token
}

# Define the parameters for the request
params = {
    'datasetid': 'GHCND',  # Global Historical Climatology Network - Daily
    'locationid': '',
    'startdate': '',
    'enddate': '',
    'limit': 1000  # Number of results to return
}

In [132]:
import time
import time
start_time = time.time()
iteration = 0
sucs = 0
fail = 0
# Iterate over the rows
for index, row in test.iterrows():
        # print('success: ', sucs, '/ fail: ', fail)
        iteration = iteration + 1
        # print(f"Iteration: {iteration}")
        if time.time() - start_time > 1800:  # 30 minutes
                print("Time limit exceeded. Stopping the process.")
                print('number of iterations: ', iteration)
                print('number of success: ', sucs)
                print('number of fail: ', fail)
                break
        params['locationid'] = row['location_id']
        params['startdate'] = row['date_ts']
        params['enddate'] = row['date_ts']
        # print(params)

        # Make the request
        try:
                response = requests.get(endpoint, headers=headers, params=params, timeout=30)
                try:
                        data = response.json()
                        # Check if 'results' key exists in the response data
                        if 'results' in data:
                                sucs = sucs + 1
                                # Insert data into the climate table
                                for record in data['results']:
                                        conn.execute('''
                                        INSERT OR REPLACE INTO climate (date, location_id, datatype, station, attributes, value)
                                        VALUES (?, ?, ?, ?, ?, ?)
                                        ''', (record['date'], row['location_id'], record['datatype'], record['station'], record['attributes'], record['value']))
                        else:
                                fail = fail + 1
                                continue
                                # conn.execute('''
                                # INSERT OR REPLACE INTO climate (date, location_id, datatype, station, attributes, value)
                                # VALUES (?, ?, ?, ?, ?, ?)
                                # ''', (row['date_ts'], row['location_id'], 'err', 'err', 'err', 'err'))
                        
                        conn.commit()

                except json.JSONDecodeError as e:
                        fail = fail + 1
                #     print(f"Error decoding JSON for location_id: {row['location_id']} on date: {row['date_ts']}")
                        continue
                #     raise e
                #     conn.execute('''
                #     INSERT OR REPLACE INTO climate (date, location_id, datatype, station, attributes, value)
                #     VALUES (?, ?, ?, ?, ?, ?)
                #     ''', (row['date_ts'], row['location_id'], 'err2', 'err2', 'err2', 'err2'))
                #     continue
        except requests.exceptions.Timeout:
                fail = fail + 1
                continue
        conn.commit()

conn.commit()

Time limit exceeded. Stopping the process.
number of iterations:  1709
number of success:  0
number of fail:  1708


### create new column for location-date index

In [127]:
print('number of iterations: ', iteration)
print('number of success: ', sucs)
print('number of fail: ', fail)

number of iterations:  5204
number of success:  0
number of fail:  5204


In [133]:
# conn.execute('''
# DELETE FROM climate
# WHERE datatype = 'err'
# ''')
conn.commit()

In [134]:
conn.execute('ALTER TABLE climate DROP COLUMN location_date_id')
conn.execute('ALTER TABLE climate ADD COLUMN location_date_id TEXT')
conn.execute('''
UPDATE climate
SET location_date_id = location_id || '_' || date(date)
''')
conn.commit()

In [11]:
pd.read_sql_query('''
-- select count(distinct location_id) as station_count
select *
FROM climate
limit 5
''', conn)

,date,location_id,datatype,station,attributes,value,location_date_id
0,2024-01-01T00:00:00,CITY:JA000016,PRCP,GHCND:JA000047662,",,S,",0.0,CITY:JA000016_2024-01-01
1,2024-01-01T00:00:00,CITY:JA000016,TAVG,GHCND:JA000047662,"H,,S,",74.0,CITY:JA000016_2024-01-01
2,2024-01-01T00:00:00,CITY:JA000016,PRCP,GHCND:JA000047670,"B,,S,",0.0,CITY:JA000016_2024-01-01
3,2024-01-01T00:00:00,CITY:JA000016,TAVG,GHCND:JA000047670,"H,,S,",82.0,CITY:JA000016_2024-01-01
4,2024-01-01T00:00:00,CITY:JA000016,TMAX,GHCND:JA000047670,",,S,",131.0,CITY:JA000016_2024-01-01


In [55]:
pd.read_sql_query('''
select count(distinct location_id) as location_count
-- select *
FROM climate
limit 5
''', conn)

,location_count
0,401


### climate data coverage test

In [45]:
pd.read_sql_query('''
with part1 as(
select distinct date, location_id, location_date_id
from results
),
part2 as(
select distinct location_date_id, 1 as climate
from climate
),
part3 as(
select date(t1.date) as date, t1.location_id, t2.climate
from part1 t1
left join part2 t2
on t1.location_date_id = t2.location_date_id
)
select count(*), sum(climate) as climate_count
from part3
-- where climate is not null
limit 3
''', conn)

,count(*),climate_count
0,36555,4727


In [71]:
pd.read_sql_query('''
select count(*),
count(case when datatype = 'err' then 1 end) as err_count
from climate
''', conn)

,count(*),err_count
0,170614,17572


In [43]:
pd.read_sql_query('''
select *
from results
where location_date_id in (select location_date_id from climate)
limit 3
''', conn)

,index,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,noaa_country_code,location_id,mindate,maxdate,date_ts,mindate_ts,maxdate_ts,valid,location_date_id
0,26,1883-03-10 00:00:00,England,Scotland,2,3,Friendly,Sheffield,United Kingdom,0,UK,CITY:UK000011,1882-12-31,2024-08-31,1883-03-10,1882-12-31,2024-08-31,1,CITY:UK000011_1883-03-10
1,49,1887-02-05 00:00:00,England,Northern Ireland,7,0,British Home Championship,Sheffield,United Kingdom,0,UK,CITY:UK000011,1882-12-31,2024-08-31,1887-02-05,1882-12-31,2024-08-31,1,CITY:UK000011_1887-02-05
2,86,1893-02-25 00:00:00,England,Northern Ireland,6,1,British Home Championship,Birmingham,United Kingdom,0,UK,CITY:UK000002,1880-12-01,2024-08-31,1893-02-25,1880-12-01,2024-08-31,1,CITY:UK000002_1893-02-25


In [77]:
lulu = pd.read_sql_query('''
with part1 as(
select strftime('%Y', date) as year,
count(*) as count,
count(case when datatype = 'err' then 1 end) as err_count
from climate
group by 1
order by 1 desc
)
select *, 1.0 * err_count / count as err_rate
from part1
''', conn)